In [1]:
from torchvision.datasets import ImageFolder

from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import SGD

import numpy as np

from config import *
from utils import test_augmentation, train_augmentation

In [2]:
# it is impossible change augmentations if ImageFolder split via random_split
train_data = ImageFolder(TRAIN_DATA, transform=train_augmentation)
valid_data = ImageFolder(TRAIN_DATA, transform=test_augmentation)

# obtain training indices
indices = np.arange(len(train_data))
np.random.shuffle(indices)
split = int(0.7 * len(train_data))
train_idx, valid_idx = indices[split:], indices[:split]

# prepare data loaders
train_loader = DataLoader(
	train_data, batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(train_idx), num_workers=12
)
valid_loader = DataLoader(
	train_data, batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(valid_idx), num_workers=12
)

In [3]:
from engine import *

In [4]:
lr = 5e-4

model = get_model().cuda()
model.train()
optimizer = SGD(model.parameters(), lr, momentum=0.9,weight_decay=1e-4)
loss_function = nn.CrossEntropyLoss()

In [5]:
model

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size

In [6]:
%%time
train_classification(
		1, model, train_loader, valid_loader, loss_function, optimizer, noti_rate=100
	)

RuntimeError: The size of tensor a (16) must match the size of tensor b (64) at non-singleton dimension 3

In [7]:
plot_result(
	*train_classification(
		50, model, train_loader, valid_loader, loss_function, optimizer, noti_rate=len(train_data) + 1
	)
)

RuntimeError: The size of tensor a (16) must match the size of tensor b (64) at non-singleton dimension 3

In [ ]:
torch.save(model, os.path.join(WEIGHT_PATH, "classification_50.model"))

In [ ]:
plot_result(
	*train_classification(
		100, model, train_loader, valid_loader, loss_function, optimizer, noti_rate=len(train_data) + 1
	)
)

In [ ]:
torch.save(model, os.path.join(WEIGHT_PATH, "classification_150.model"))

In [ ]:
plot_result(
	*train_classification(
		550, model, train_loader, valid_loader, loss_function, optimizer, noti_rate=len(train_data) + 1
	)
)

In [ ]:
torch.save(model, os.path.join(WEIGHT_PATH, "classification_700.model"))

In [ ]:
plot_result(
	*train_classification(
		1000, model, train_loader, valid_loader, loss_function, optimizer, noti_rate=len(train_data) + 1
	)
)

In [ ]:
torch.save(model, os.path.join(WEIGHT_PATH, "classification_1000.model"))